In [4]:
import asyncio
from apify_client import ApifyClient
import json
import os
from datetime import datetime, timedelta
import pytz

# Initialize the ApifyClient with your API token
client = ApifyClient("apify_api_eK6nKjS5fr6yrbLoREGXRUMQ0UrF1Z20oWFt")

def save_data_to_json(data, file_name):
    if not os.path.exists(file_name):
        # If the file does not exist, create a new one and write the data as a list
        with open(file_name, 'w') as file:
            json.dump([data], file, indent=4)
    else:
        # If the file exists, load the existing data
        with open(file_name, 'r+') as file:
            try:
                existing_data = json.load(file)
            except json.JSONDecodeError:
                existing_data = []
            
            # Append the new data to the list
            existing_data.append(data)
            
            # Set the file cursor to the beginning, truncate the file, and save the updated list
            file.seek(0)
            json.dump(existing_data, file, indent=4)
            file.truncate()

    print(f"Data saved to {file_name}")


def extract_recent_urls(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    urls = []
    current_date = datetime.now()
    one_week_ago = current_date - timedelta(weeks=1)
    
    for post in data:
        created_at_str = post.get("created_at")
        created_at = datetime.strptime(created_at_str, "%Y-%m-%d %H:%M:%S")
        
        if created_at >= one_week_ago:
            url = post.get("url")
            if url:
                urls.append(url)
    
    return urls

# Initialize an empty set to track seen posts
seen_comments = set()

seconds_for_next_run = 300  # 5 minutes

async def fetch_instagram_posts():
    # Define the maximum number of posts to retrieve
    results = 50

    # Determine the URLs of the most recent Instagram posts
    urls = extract_recent_urls('instagram_posts.json')
    print(f"Extracted {len(urls)} recent Instagram posts")

    run_input = {
        "addParentData": False,
        "directUrls": urls,
        "enhanceUserSearchWithFacebookPage": False,
        "isUserReelFeedURL": False,
        "isUserTaggedFeedURL": False,
        "resultsLimit": results,
        "resultsType": "comments",
        "searchLimit": 1,
        "searchType": "hashtag"
    }

    # Run the Actor and wait for it to finish
    run = client.actor("shu8hvrXbJbY3Eb9W").call(run_input=run_input)

    # Fetch and print Actor results from the run's dataset (if there are any)
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        comment_id = item.get('id')

        # Check if the post has already been seen
        if comment_id not in seen_comments:
           
            post_comments = {}

            created_at = item.get('timestamp')

            # Parse the timestamp as a datetime object in UTC
            utc_time = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%S.%fZ")

            # Define the timezone for Guayaquil, Ecuador (which is UTC-5)
            guayaquil_tz = pytz.timezone('America/Guayaquil')

            # Convert the UTC time to Guayaquil time
            guayaquil_time = utc_time.replace(tzinfo=pytz.utc).astimezone(guayaquil_tz)

            guayaquil_time_str = guayaquil_time.strftime("%Y-%m-%d %H:%M:%S")

            post_url = item.get('postUrl')
            comment = item.get('text')
            user = item.get('ownerUsername')
            user_profile = item.get('ownerProfilePicUrl')
            created_at = guayaquil_time_str
            likes_count = item.get('likesCount')
            

            post_comments['postUrl'] = post_url
            post_comments['comment_id'] = comment_id
            post_comments['comment'] = comment
            post_comments['user'] = user
            post_comments['user_profile'] = user_profile
            post_comments['created_at'] = created_at
            post_comments['likes_count'] = likes_count
            

            # Add the post ID to the seen_posts set
            seen_comments.add(comment_id)
            
            print(f"Comment {[comment_id]} extracted")

            save_data_to_json(post_comments, 'instagram_comments.json')

        else:
            print(f"Comment {comment_id} already extracted sometime ago")

async def main():
    while True:
        await fetch_instagram_posts()
        print("Waiting for 5 minutes before the next execution...")
        await asyncio.sleep(seconds_for_next_run)  # Sleep for 2 minutes (120 seconds)

if __name__ == "__main__":
    asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [3]:
from apify_client import ApifyClient

# Initialize the ApifyClient with your API token
client = ApifyClient("apify_api_eK6nKjS5fr6yrbLoREGXRUMQ0UrF1Z20oWFt")

# Prepare the Actor input
run_input = {
    "addParentData": False,
    "directUrls": [
        "'https://www.instagram.com/p/C-Y4jTARMAh/",
        "https://www.instagram.com/p/C-Y4gWSx5vV/",
        "https://www.instagram.com/p/C-Y4vPiRKek/",
        "https://www.instagram.com/p/C-VTs0URuP3/"
    ],
    "enhanceUserSearchWithFacebookPage": False,
    "isUserReelFeedURL": False,
    "isUserTaggedFeedURL": False,
    "resultsLimit": 10,
    "resultsType": "comments",
    "searchLimit": 1,
    "searchType": "hashtag"
}

# Run the Actor and wait for it to finish
run = client.actor("shu8hvrXbJbY3Eb9W").call(run_input=run_input)

# Fetch and print Actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item)

{'postUrl': 'https://www.instagram.com/p/C-VTs0URuP3/', 'id': '18247760392264306', 'text': 'Lo máximo!! Gracias por ser parte 🎉😊', 'ownerUsername': 'danicostawave', 'ownerProfilePicUrl': 'https://scontent-lhr8-1.cdninstagram.com/v/t51.2885-19/306912692_1122916868596447_1563862323867436199_n.jpg?stp=dst-jpg_s150x150&_nc_ht=scontent-lhr8-1.cdninstagram.com&_nc_cat=107&_nc_ohc=RCDwxErRK08Q7kNvgHQZkCz&edm=AP_V10EBAAAA&ccb=7-5&oh=00_AYAwGD3DbmxWN3I5gvJeQhj3jysws4JDHAy4hV0QCTa6kQ&oe=66BC337F&_nc_sid=2999b8', 'timestamp': '2024-08-06T15:34:05.000Z', 'likesCount': 2}
{'postUrl': 'https://www.instagram.com/p/C-VTs0URuP3/', 'id': '18059157700582114', 'text': '👏🔥', 'ownerUsername': 'mhedu_eb', 'ownerProfilePicUrl': 'https://scontent-lhr8-1.cdninstagram.com/v/t51.2885-19/54512400_257811048490616_4989889345180663808_n.jpg?stp=dst-jpg_s150x150&_nc_ht=scontent-lhr8-1.cdninstagram.com&_nc_cat=108&_nc_ohc=3yV6uoAMDKYQ7kNvgHtSB8V&edm=AP_V10EBAAAA&ccb=7-5&oh=00_AYCjjKVMculxsMCUnXQxdLpe7MQO9zGAUvwDPRIxz2c